In [ ]:
#!/usr/bin/env python3

import rclpy
import numpy as np
from robot_interface_client import RobotInterfaceClient


def main():
    rclpy.init()
    robot = RobotInterfaceClient()

    print("⏳ Waiting for FK data...")

    try:
        while rclpy.ok():
            rclpy.spin_once(robot, timeout_sec=0.1)

            orient = robot.current_orientation
            if orient is not None:
                break

        print("\n===== FK Orientation Check =====")
        print("orient value:", orient)
        print("length:", len(orient))
        print("norm:", np.linalg.norm(orient))

        if len(orient) == 4:
            print("✅ Quaternion (x, y, z, w)")
        elif len(orient) == 3:
            print("⚠ Euler angles (roll, pitch, yaw)")
        else:
            print("❌ Unknown orientation format")

    finally:
        rclpy.shutdown()


if __name__ == "__main__":
    main()


In [1]:
#!/usr/bin/env python3

import rclpy
import numpy as np
from rclpy.node import Node

import tf2_ros
from tf2_ros import TransformException


class EEPoseListener(Node):
    def __init__(self):
        super().__init__("ee_pose_listener")

        # TF buffer & listener
        self.tf_buffer = tf2_ros.Buffer()
        self.tf_listener = tf2_ros.TransformListener(self.tf_buffer, self)

        # ⚠️ 프레임 이름 확인해서 필요하면 수정
        self.base_frame = "world"
        self.ee_frame = "end_effector_link"

        self.get_logger().info("⏳ Waiting for TF data (EE pose)...")

    def get_ee_pose(self):
        try:
            tf = self.tf_buffer.lookup_transform(
                self.base_frame,
                self.ee_frame,
                rclpy.time.Time()
            )

            t = tf.transform.translation
            q = tf.transform.rotation

            position = np.array([t.x, t.y, t.z])
            orientation = np.array([q.x, q.y, q.z, q.w])

            return position, orientation

        except TransformException as e:
            return None, None


def main():
    rclpy.init()
    node = EEPoseListener()

    try:
        while rclpy.ok():
            rclpy.spin_once(node, timeout_sec=0.1)

            pos, quat = node.get_ee_pose()

            if pos is not None:
                break

        print("\n===== EE Pose from TF =====")
        print(f"Position (x,y,z): {pos}")
        print(f"Orientation (qx,qy,qz,qw): {quat}")
        print("Quaternion norm:", np.linalg.norm(quat))

    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == "__main__":
    main()



===== EE Pose from TF =====
Position (x,y,z): [ 1.61726811e-01 -1.23642380e-05  2.02504519e-01]
Orientation (qx,qy,qz,qw): [-1.94714667e-09 -4.71585978e-05 -4.12893249e-05  9.99999998e-01]
Quaternion norm: 0.9999999999999999


[INFO] [1767492459.427235096] [ee_pose_listener]: ⏳ Waiting for TF data (EE pose)...


In [2]:
#!/usr/bin/env python3

import rclpy
from robot_interface_client import RobotInterfaceClient

def main():
    rclpy.init()
    robot = RobotInterfaceClient()

    # 1️⃣ 반드시 초기자세
    robot.call_move_to_named("home")

    # 2️⃣ EE가 충분히 도달 가능한 좌표
    x = 0.16
    y = 0.0
    z = 0.20

    # orientation은 의미 없음
    robot.call_move_to_pose(
        x, y, z,
        0.0, 0.0, 0.0, 1.0
    )

    rclpy.shutdown()

if __name__ == "__main__":
    main()


[INFO] [1767492653.426157304] [robot_interface_client]: 🧪 robot_interface_client Node initialized
[INFO] [1767492653.463339164] [robot_interface_client]: ⏳ Waiting for all service servers...


KeyboardInterrupt: 